In [21]:
from codebook import Codebook

vq = Codebook(
    input_size=4,
    embedding_size=4,
    num_embedding=4,
    use_cosine_sim=True
)

import torch
T = 5
x = torch.randn(2, T, 4)

In [36]:
code = vq.quantize_to_code(x)

code.shape, code

code = torch.tensor([[0,1,1,3],[0,1,2,3]])
code1 = torch.tensor([[0,0,1,3],[0,1,2,3]])

In [41]:
F.one_hot(code, num_classes=4).view(-1,4).sum(0) + F.one_hot(code1, num_classes=4).view(-1,4).sum(0)

tensor([5, 5, 2, 4])

In [38]:
F.one_hot(code1, num_classes=4).view(-1,4).float().mean(0)

tensor([0.3750, 0.2500, 0.1250, 0.2500])

In [35]:
import torch.nn.functional as F
e_mean = F.one_hot(code, num_classes=4).view(-1, 4).float().mean(0)
e_mean

tensor([0.2500, 0.3750, 0.1250, 0.2500])

In [42]:
F.one_hot(code, num_classes=4).view(-1, 4).float()

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [29]:
torch.stack([torch.randn(2,10), torch.randn(2,10), torch.randn(2,10)], dim=1).shape

torch.Size([2, 3, 10])

In [26]:
perplexity = torch.exp(-torch.sum(e_mean * torch.log(e_mean + 1e-10)))
perplexity

tensor(4.)

In [28]:
one_hot = torch.nn.functional.one_hot(code, num_classes=1024)
counts = one_hot.sum(dim=1)
counts / T

tensor([[0.2000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2000,  ..., 0.0000, 0.0000, 0.0000]])

In [17]:
dist / (T*2)

tensor([0.1000, 0.0000, 0.1000, 0.0500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0500,
        0.0500, 0.0500, 0.0000, 0.0000, 0.0500, 0.0000, 0.0500, 0.0000, 0.0000,
        0.0000, 0.0000, 0.1000, 0.0000, 0.1000, 0.0000, 0.0500, 0.0500, 0.0000,
        0.0500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0500, 0.0500, 0.0000,
        0.0000, 0.0500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [22]:
dec = torch.randn(4, 2, 2)
residual_q = torch.ones(4,2,2)
mask = (
                torch.full((dec.shape[0],), fill_value=False, device=dec.device)
            )
residual_q *= mask[:, None, None]
dec

tensor([[[ 0.0040,  0.9420],
         [ 0.5901, -1.5170]],

        [[-1.7792, -0.1886],
         [-0.5103,  0.1227]],

        [[-1.1521, -0.6097],
         [ 0.8428,  1.3517]],

        [[-1.0177,  1.8646],
         [ 2.5531, -0.0660]]])

In [23]:
residual_q

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]])

In [24]:
torch.nn.Identity()(residual_q + dec)

tensor([[[ 0.0040,  0.9420],
         [ 0.5901, -1.5170]],

        [[-1.7792, -0.1886],
         [-0.5103,  0.1227]],

        [[-1.1521, -0.6097],
         [ 0.8428,  1.3517]],

        [[-1.0177,  1.8646],
         [ 2.5531, -0.0660]]])

In [1]:
from finite_scalar_quantize import FSQ
fsq = FSQ(levels=[2]*2, num_codebooks=1, dim=6, )

In [2]:
fsq.training

True

In [3]:
fsq.eval()
fsq.training

False

In [4]:
import torch
x = torch.randn(1, 2, 6)
x

tensor([[[-0.2831,  0.1810,  1.0986, -1.2818,  0.7840, -1.2412],
         [ 0.0375, -0.7446,  0.9225, -1.0288,  0.2784, -0.0822]]])

In [5]:
out, indices = fsq(x)
print(out.shape, indices.shape)
out, indices

torch.Size([1, 2, 6]) torch.Size([1, 2])


(tensor([[[ 0.4075,  0.2481,  0.1417,  0.4198, -0.5422, -0.1975],
          [ 0.4075,  0.2481,  0.1417,  0.4198, -0.5422, -0.1975]]],
        grad_fn=<ViewBackward0>),
 tensor([[3, 3]], dtype=torch.int32))

In [6]:
fsq.train()
print(fsq.training)

out, indices = fsq(x)
print(out.shape, indices.shape)
out, indices

True
torch.Size([1, 2, 6]) torch.Size([1, 2])


(tensor([[[ 0.4075,  0.2481,  0.1417,  0.4198, -0.5422, -0.1975],
          [ 0.4075,  0.2481,  0.1417,  0.4198, -0.5422, -0.1975]]],
        grad_fn=<ViewBackward0>),
 tensor([[3, 3]], dtype=torch.int32))